In [1]:
import org.apache.spark.sql.functions.{explode, split, col}
import org.apache.spark.sql.SparkSession

In [2]:
val spark = SparkSession.builder()
  .appName("Remove Duplicate Movies")
  .master("local[*]") // Use appropriate cluster manager in production
  .getOrCreate()

spark = org.apache.spark.sql.SparkSession@3e5ef5f8


org.apache.spark.sql.SparkSession@3e5ef5f8

In [3]:
import spark.implicits._

In [4]:
val moviesDffPath = "gs://artifacts_spark_jobs/movie.csv"

val moviesDff = spark.read.format("csv").option("header","true").option("inferSchema","true").load(moviesDffPath)

moviesDffPath = gs://artifacts_spark_jobs/movie.csv
moviesDff = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [5]:
val moviesRDD = moviesDff.rdd

moviesRDD = MapPartitionsRDD[14] at rdd at <console>:30


MapPartitionsRDD[14] at rdd at <console>:30

In [6]:
val distinctRdd = moviesRDD.map(row => ((row.getInt(0), row.getString(1)), row)) // Create composite key
  .reduceByKey((x, _) => x)  // Retain only one record per composite key
  .map(_._2)  

distinctRdd = MapPartitionsRDD[17] at map at <console>:32


MapPartitionsRDD[17] at map at <console>:32

In [7]:
distinctRdd.count()

27278

In [22]:
moviesDff.count() // count of all

27278

In [24]:
// Add depedencies for avro format
libraryDependencies += "org.apache.spark" %% "spark-avro" % "3.0.1"

val hdfsPath = "hdfs:/user/hdfs/CaseStudies"

Unknown Error: <console>:62: error: not found: value libraryDependencies
       libraryDependencies += "org.apache.spark" %% "spark-avro" % "3.0.1"
       ^
<console>:62: error: value %% is not a member of String
       libraryDependencies += "org.apache.spark" %% "spark-avro" % "3.0.1"
                                                 ^


In [ ]:
val gsPath = "gs://artifacts_spark_jobs/yaseen/movie-clean-data"


distinctRdd.map {row =>
    val movieId = row.getAs[Int]("movieId")
    val title = row.getAs[String]("title")
    val genres = row.getAs[String]("genres")
    
    (movieId, title, genres)
}.toDF("movieId", "title", "genres")
.write
.format("avro")
.mode("overwrite")
.save(gsPath)